# Data loading 
Next cells are used to load the data we need.

In [ ]:
!git clone #clone your repo here
!mv Recommender-Systems-Challenge/* ../content/

Cloning into 'Recommender-Systems-Challenge'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 1082 (delta 92), reused 89 (delta 48), pack-reused 937
Receiving objects: 100% (1082/1082), 208.66 MiB | 38.14 MiB/s, done.
Resolving deltas: 100% (630/630), done.


In [ ]:
!python run_compile_all_cython.py

run_compile_all_cython: Found 11 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/MatrixFactorization/Cython/MatrixFactorization_Cython_Epoch.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
In file included from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1822:0,
                 from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorization_Cython_Epoch.c:610:
/usr/local/lib/python3.6/dist-packages/numpy/core/include/numpy/npy_1_7_d

In [ ]:
pip install implicit

     |████████████████████████████████| 1.1MB 12.1MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419409 sha256=01d8968aa7d8e4df23ff11f64b574308dc7fe7775db6501d6943b2a3b9778fc0
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [ ]:
pip install optuna

     |████████████████████████████████| 266kB 12.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 9.7MB/s 
     |████████████████████████████████| 163kB 44.4MB/s 
     |████████████████████████████████| 112kB 48.6MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 133kB 46.7MB/s 
     |████████████████████████████████| 81kB 8.8MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-cp36-none-any.whl size=359761 sha256=2d7903a180fd6de074548111c2aaeb5cdfd0885df4b4d2cc69cbd2c52a06a0f8
  Stored in directory: /root/.cache/pip/wheels/fa/91/19/64b0ec6b964f89c0695a9dc6db6f851d0b54c5381a5c9cadfb
Successfully built optuna
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp36-none-any.whl size=13702 sha256=76bc73ed60d0a4f49479ab605ab5b2e17f2ab421dd34aff1690b2c9151b56576
  Stored in directory: /root/.cache/pip/wh

In [ ]:
import os
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import sys
import matplotlib.pyplot as plt
import optuna


from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from GraphBased.RP3betaRecommender import RP3betaRecommender

In [ ]:
dataset = pd.read_csv('/content/New Challenge Data/data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [ ]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

test_users = pd.read_csv('/content/New Challenge Data/data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
7939,7942
7940,7943
7941,7944
7942,7945


In [ ]:
ICM_df = pd.read_csv('/content/New Challenge Data/data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [ ]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [ ]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Base.Evaluation.Evaluator import EvaluatorHoldout

URM_train_1, URM_validation_1 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 1)

URM_train_2, URM_validation_2 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 2)

URM_train_3, URM_validation_3 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 3)

URM_train_4, URM_validation_4 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 4)

URM_train_5, URM_validation_5 = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed = 5)

URM_train_vector = [URM_train_1,URM_train_2,URM_train_3,URM_train_4,URM_train_5]

evaluator_validation_1 = EvaluatorHoldout(URM_validation_1, cutoff_list=[10])
evaluator_validation_2 = EvaluatorHoldout(URM_validation_2, cutoff_list=[10])
evaluator_validation_3 = EvaluatorHoldout(URM_validation_3, cutoff_list=[10])
evaluator_validation_4 = EvaluatorHoldout(URM_validation_4, cutoff_list=[10])
evaluator_validation_5 = EvaluatorHoldout(URM_validation_5, cutoff_list=[10])

Evaluator_vector = [evaluator_validation_1, evaluator_validation_2, evaluator_validation_3, evaluator_validation_4, evaluator_validation_5]

In [ ]:
class Objective(object):
    def __init__(self, URM_train_vector, ICM_all, Evaluator_vector):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train_vector = URM_train_vector
        self.ICM_all = ICM_all
        self.Evaluator_vector = Evaluator_vector
        self.MAP_vector = np.zeros(5)

        

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        topK = trial.suggest_int('topK', 150,300)
        alpha = trial.suggest_uniform('alpha', 0.3,0.45)
        beta = trial.suggest_uniform('beta', 0.2,0.35)
        #weight = trial.suggest_uniform('weight', 0.1, 0.9)
        self.MAP_vector = np.zeros(5)
        i = 0
        for URM_train, evaluator_validation in zip(self.URM_train_vector,self.Evaluator_vector):
            URM_train_ICM_all = sps.vstack([URM_train, ICM_all.T])
            recommender = RP3betaRecommender(URM_train_ICM_all)
            recommender.fit(topK = topK, alpha = alpha, beta = beta, implicit = False)
            result_dict, _ = evaluator_validation.evaluateRecommender(recommender)
            self.MAP_vector[i]=result_dict[10]["MAP"]
            i = i+1
            
        print('printing self map vector: ', self.MAP_vector)
        MAP = np.sum(self.MAP_vector)/5
        print('printing MAP: ', MAP)
        
        return MAP
    
# Execute an optimization by using an `Objective` instance.
study = optuna.create_study(direction='maximize')
study.optimize(Objective(URM_train_vector, ICM_all, Evaluator_vector), n_trials=70)

print(study.best_params)

[I 2021-01-05 14:52:45,076] A new study created in memory with name: no-name-866ffab3-be0f-4dba-aa82-a6cd4bdd1504


RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
EvaluatorHoldout: Processed 5536 ( 100.00% ) in 3.28 sec. Users per second: 1690
RP3betaRecommender: URM Detected 59 (0.21 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
EvaluatorHoldout: Processed 5627 ( 100.00% ) in 3.28 sec. Users per second: 1716
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 4 (0.02 %) cold items.
EvaluatorHoldout: Processed 5658 ( 100.00% ) in 3.30 sec. Users per second: 1713
RP3betaRecommender: URM Detected 76 (0.27 %) cold users.
EvaluatorHoldout: Processed 5674 ( 100.00% ) in 3.26 sec. Users per second: 1740
RP3betaRecommender: URM Detected 66 (0.24 %) cold users.
RP3betaRecommender: URM Detected 3 (0.01 %) cold items.
EvaluatorHoldout: Processed 5589 ( 100.00% ) in 3.22 sec. Users per second: 1735


[I 2021-01-05 14:55:44,241] Trial 0 finished with value: 0.07281391959242267 and parameters: {'topK': 283, 'alpha': 0.32590005552017204, 'beta': 0.32161100520842023}. Best is trial 0 with value: 0.07281391959242267.


printing self map vector:  [0.07978782 0.07179201 0.07205365 0.0718081  0.06862802]
printing MAP:  0.07281391959242267
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
EvaluatorHoldout: Processed 5536 ( 100.00% ) in 3.06 sec. Users per second: 1809
RP3betaRecommender: URM Detected 59 (0.21 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
EvaluatorHoldout: Processed 5627 ( 100.00% ) in 3.08 sec. Users per second: 1829
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 4 (0.02 %) cold items.
EvaluatorHoldout: Processed 5658 ( 100.00% ) in 3.14 sec. Users per second: 1801
RP3betaRecommender: URM Detected 76 (0.27 %) cold users.
EvaluatorHoldout: Processed 5674 ( 100.00% ) in 3.05 sec. Users per second: 1858
RP3betaRecommender: URM Detected 66 (0.24 %) cold users.
RP3betaRecommender: URM Detected 3 (0.01 %) cold items.
EvaluatorHoldout: Processed 5589 ( 100.00% ) in 

[I 2021-01-05 14:58:26,526] Trial 1 finished with value: 0.07343935676438407 and parameters: {'topK': 193, 'alpha': 0.3666440792749897, 'beta': 0.2502533561164425}. Best is trial 1 with value: 0.07343935676438407.


...

printing self map vector:  [0.08017812 0.07359717 0.07257533 0.07201179 0.06900012]
printing MAP:  0.0734725060935315
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
EvaluatorHoldout: Processed 5536 ( 100.00% ) in 2.84 sec. Users per second: 1948
RP3betaRecommender: URM Detected 59 (0.21 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
EvaluatorHoldout: Processed 5627 ( 100.00% ) in 2.84 sec. Users per second: 1980
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 4 (0.02 %) cold items.
EvaluatorHoldout: Processed 5658 ( 100.00% ) in 2.87 sec. Users per second: 1973
RP3betaRecommender: URM Detected 76 (0.27 %) cold users.
EvaluatorHoldout: Processed 5674 ( 100.00% ) in 2.88 sec. Users per second: 1970
RP3betaRecommender: URM Detected 66 (0.24 %) cold users.
RP3betaRecommender: URM Detected 3 (0.01 %) cold items.
EvaluatorHoldout: Processed 5589 ( 100.00% ) in 2

[I 2021-01-05 17:57:22,440] Trial 68 finished with value: 0.07323785995816558 and parameters: {'topK': 171, 'alpha': 0.38112459730166526, 'beta': 0.25976504641921155}. Best is trial 32 with value: 0.07365058857470327.


printing self map vector:  [0.07981142 0.0731304  0.07207217 0.07254002 0.06863528]
printing MAP:  0.07323785995816558
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
EvaluatorHoldout: Processed 5536 ( 100.00% ) in 2.83 sec. Users per second: 1957
RP3betaRecommender: URM Detected 59 (0.21 %) cold users.
RP3betaRecommender: URM Detected 1 (0.00 %) cold items.
EvaluatorHoldout: Processed 5627 ( 100.00% ) in 2.88 sec. Users per second: 1956
RP3betaRecommender: URM Detected 73 (0.26 %) cold users.
RP3betaRecommender: URM Detected 4 (0.02 %) cold items.
EvaluatorHoldout: Processed 5658 ( 100.00% ) in 2.90 sec. Users per second: 1951
RP3betaRecommender: URM Detected 76 (0.27 %) cold users.
EvaluatorHoldout: Processed 5674 ( 100.00% ) in 2.89 sec. Users per second: 1965
RP3betaRecommender: URM Detected 66 (0.24 %) cold users.
RP3betaRecommender: URM Detected 3 (0.01 %) cold items.
EvaluatorHoldout: Processed 5589 ( 100.00% ) in 

[I 2021-01-05 17:59:55,462] Trial 69 finished with value: 0.07354702782737593 and parameters: {'topK': 197, 'alpha': 0.37626616395090473, 'beta': 0.27165093674316015}. Best is trial 32 with value: 0.07365058857470327.


printing self map vector:  [0.07976439 0.0736364  0.07247344 0.07247256 0.06938834]
printing MAP:  0.07354702782737593
{'topK': 207, 'alpha': 0.36128728066686633, 'beta': 0.26052832791376884}
